# Phase 2: Data Preprocessing

**Objective**: Clean, transform, and merge collected data for analysis

## Tasks:
1. Load raw data files
2. Handle missing values
3. Remove outliers
4. Filter date ranges (2021-2023)
5. Aggregate to monthly level
6. Merge datasets
7. Save processed data

## Input Files:
- `data/raw/nyc_consumption.csv`
- `data/raw/eia_generation.csv`
- `data/raw/noaa_weather.csv`
- `data/raw/census_population.csv`
- `data/raw/electricity_rates.csv`

## Output Files:
- `data/processed/consumption_clean.csv`
- `data/processed/generation_clean.csv`
- `data/processed/weather_clean.csv`
- `data/processed/merged_data.csv`

In [88]:
import pandas as pd
import numpy as np
from datetime import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

In [89]:
print("="*60)
print("1. LOADING NYC CONSUMPTION DATA")
print("="*60)

# Load data
df_consumption = pd.read_csv('../data/raw/nyc_consumption.csv')

print(f"\nOriginal data shape: {df_consumption.shape}")
print(f"Columns: {list(df_consumption.columns)}")
print(df_consumption.head())

1. LOADING NYC CONSUMPTION DATA

Original data shape: (521035, 27)
Columns: ['Development Name', 'Borough', 'Account Name', 'Location', 'Meter AMR', 'Meter Scope', 'TDS #', 'EDP', 'RC Code', 'Funding Source', 'AMP #', 'Vendor Name', 'UMIS BILL ID', 'Revenue Month', 'Service Start Date', 'Service End Date', '# days', 'Meter Number', 'Estimated', 'Current Charges', 'Rate Class', 'Bill Analyzed', 'Consumption (KWH)', 'KWH Charges', 'Consumption (KW)', 'KW Charges', 'Other charges']
  Development Name Borough Account Name Location Meter AMR   Meter Scope  \
0            ADAMS   BRONX        ADAMS   BLD 05      NONE  BLD 01 to 07   
1            ADAMS   BRONX        ADAMS   BLD 05      NONE  BLD 01 to 07   
2            ADAMS   BRONX        ADAMS   BLD 05      NONE  BLD 01 to 07   
3            ADAMS   BRONX        ADAMS   BLD 05      NONE  BLD 01 to 07   
4            ADAMS   BRONX        ADAMS   BLD 05      NONE  BLD 01 to 07   

   TDS #  EDP  RC Code Funding Source  ... Meter Number Est

In [90]:
print("\n" + "="*60)
print("CLEANING CONSUMPTION DATA")
print("="*60)

# 1. Select necessary columns 
columns_needed = ['Borough','Revenue Month','Consumption (KWH)']

# Adjust based on actual column names
df_consumption_clean = df_consumption[columns_needed].copy()

# 2. Rename columns for simplicity
df_consumption_clean.columns = ['Borough','Date','Consumption_KWH']

# 3. Convert date format 
df_consumption_clean['Date'] = pd.to_datetime(
    df_consumption_clean['Date'],
    format = '%Y-%m', # Adjust if format differs
    errors = 'coerce'
)
    
# 4. Filter for 2021 - 2023 only 
df_consumption_clean = df_consumption_clean[
    (df_consumption_clean['Date'] >= '2021-01-01') & 
    (df_consumption_clean['Date'] <= '2023-12-31')
]

# 5. Calculate monthly sum by Borough (remove duplicates)
df_consumption_clean = df_consumption_clean.groupby(
    ['Borough', 'Date']
)['Consumption_KWH'].sum().reset_index()


print(df_consumption_clean.shape)
print(df_consumption_clean.head(10))


CLEANING CONSUMPTION DATA
(252, 3)
  Borough       Date  Consumption_KWH
0   BRONX 2021-01-01       24747982.0
1   BRONX 2021-02-01       21503857.0
2   BRONX 2021-03-01       20692175.0
3   BRONX 2021-04-01       21424289.0
4   BRONX 2021-05-01       20748428.0
5   BRONX 2021-06-01       26845332.0
6   BRONX 2021-07-01       33006942.0
7   BRONX 2021-08-01       36462677.0
8   BRONX 2021-09-01       29853273.0
9   BRONX 2021-10-01       23251802.0


In [91]:
print("\n" + "="*60)
print("HANDLING MISSING VALUES")
print("="*60)

# Check missing values
missing_count = df_consumption_clean.isnull().sum()
print(f"Missing value counts:\n{missing_count}")

# Final check
print("Cleaning complete:")
print(df_consumption_clean.info())

# Save cleaned data 
output_path = '../data/processed/consumption_clean.csv'
df_consumption_clean.to_csv(output_path, index=False)
print(f"Saved to: {output_path}")


HANDLING MISSING VALUES
Missing value counts:
Borough            0
Date               0
Consumption_KWH    0
dtype: int64
Cleaning complete:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Borough          252 non-null    object        
 1   Date             252 non-null    datetime64[ns]
 2   Consumption_KWH  252 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 6.0+ KB
None
Saved to: ../data/processed/consumption_clean.csv


In [62]:
print("\n" + "="*60)
print("2. LOADING EIA GENERATION DATA")
print("="*60)

# Load data
df_generation = pd.read_csv('../data/raw/eia_generation.csv',sep=';')


print(f"\nOriginal data shape: {df_generation.shape}")
print(f"\nColumns: {list(df_generation.columns)}")
print("\nFirst 10 rows:")
print(df_generation.head(10))


# Check column names
print("\nColumn names:")
print(df_generation.columns.tolist())


2. LOADING EIA GENERATION DATA

Original data shape: (15, 1)

Columns: ['Net generation for electric power']

First 10 rows:
                   Net generation for electric power
0  https://www.eia.gov/electricity/data/browser/#...
1  Thu Oct 02 2025 13:14:38 GMT-0400 (Eastern Day...
2     Source: U.S. Energy Information Administration
3  description,"units","source key","Jan 2021","F...
4  Net generation for electric power,"thousand me...
5  United States,,"ELEC.GEN..M",,,,,,,,,,,,,,,,,,...
6  Middle Atlantic,,"ELEC.GEN..M",,,,,,,,,,,,,,,,...
7  New York,,"ELEC.GEN..M",,,,,,,,,,,,,,,,,,,,,,,...
8  New York : all fuels (utility-scale),"thousand...
9  New York : conventional hydroelectric,"thousan...

Column names:
['Net generation for electric power']


In [23]:
print("\n" + "="*60)
print("PROCESSING GENERATION DATA")
print("="*60)

# This is a wide format that needs to be converted to long format
# check first few columns to understand structure
print("\nFirst column values (to find energy source labels):")
print(df_generation.iloc[:, 0].head(10))

# we need to identify:
# 1. Row containing energy source names
# 2. Date columns 

# Find the row index that contains resource labels
# Usually it's in a column like 'description' or similar
print("First column values")
print(df_generation.iloc[:,0].head(10))


PROCESSING GENERATION DATA

First column values (to find energy source labels):
0    https://www.eia.gov/electricity/data/browser/#...
1    Thu Oct 02 2025 13:14:38 GMT-0400 (Eastern Day...
2       Source: U.S. Energy Information Administration
3    description,"units","source key","Jan 2021","F...
4    Net generation for electric power,"thousand me...
5    United States,,"ELEC.GEN..M",,,,,,,,,,,,,,,,,,...
6    Middle Atlantic,,"ELEC.GEN..M",,,,,,,,,,,,,,,,...
7    New York,,"ELEC.GEN..M",,,,,,,,,,,,,,,,,,,,,,,...
8    New York : all fuels (utility-scale),"thousand...
9    New York : conventional hydroelectric,"thousan...
Name: Net generation for electric power, dtype: object
First column values
0    https://www.eia.gov/electricity/data/browser/#...
1    Thu Oct 02 2025 13:14:38 GMT-0400 (Eastern Day...
2       Source: U.S. Energy Information Administration
3    description,"units","source key","Jan 2021","F...
4    Net generation for electric power,"thousand me...
5    United States,

In [28]:
print("\n" + "="*60)
print("2.1 LOADING EIA GENERATION DATA (SKIP HEADER ROWS)")
print("="*60)

# EIA files typically have metadata in first few rows
# Try skipping first 4 rows
for skip_rows in [0, 1, 2, 3, 4, 5]:
    try:
        print(f"\nTrying with skiprows={skip_rows}...")
        df_generation = pd.read_csv(
            '../data/raw/eia_generation.csv',
            skiprows=skip_rows
        )
        print(f"Success! Shape: {df_generation.shape}")
        
        if df_generation.shape[1] > 5:  # If we have multiple columns
            print(f"Columns (first 10): {df_generation.columns[:10].tolist()}")
            print("\nFirst 3 rows:")
            print(df_generation.head(3))
            break
    except Exception as e:
        print(f"Failed: {str(e)[:100]}")

print(f"\nFinal loaded shape: {df_generation.shape}")


2.1 LOADING EIA GENERATION DATA (SKIP HEADER ROWS)

Trying with skiprows=0...
Failed: Error tokenizing data. C error: Expected 1 fields in line 5, saw 39


Trying with skiprows=1...
Failed: Error tokenizing data. C error: Expected 1 fields in line 5, saw 39


Trying with skiprows=2...
Failed: Error tokenizing data. C error: Expected 1 fields in line 5, saw 39


Trying with skiprows=3...
Success! Shape: (12, 1)

Trying with skiprows=4...
Success! Shape: (11, 39)
Columns (first 10): ['description', 'units', 'source key', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021', 'Jul 2021']

First 3 rows:
                         description                   units   source key  \
0  Net generation for electric power  thousand megawatthours          NaN   
1                      United States                     NaN  ELEC.GEN..M   
2                    Middle Atlantic                     NaN  ELEC.GEN..M   

   Jan 2021  Feb 2021  Mar 2021  Apr 2021  May 2021  Jun 2021  Jul

In [31]:
print("\n" + "="*60)
print("STEP 2.2: EXTRACT NEW YORK ENERGY DATA")
print("="*60)

# We need to find rows containing New York data for:
# - All fuels (total generation)
# - Wind
# - Solar  
# - Conventional Hydroelectric

# Check which rows contain "New York"
print("Rows containing 'New York':")
ny_mask = df_generation['description'].str.contains('New York', case=False, na=False)
print(df_generation[ny_mask]['description'].tolist())


STEP 2.2: EXTRACT NEW YORK ENERGY DATA
Rows containing 'New York':
['New York', 'New York : all fuels (utility-scale)', 'New York : conventional hydroelectric', 'New York : other renewables', 'New York : wind', 'New York : all utility-scale solar', 'New York : all solar', 'New York : all utility-scale solar']


In [34]:
# Find specific energy sources for New York 
def find_ny_energy_row(df, energy_type):
    for idx, row in df.iterrows():
        desc = str(row['description']).lower()
        if 'new york' in desc and energy_type.lower() in desc:
            return idx
    return None

# Find row indices 
all_fuels_idx = find_ny_energy_row(df_generation, 'all fuels')
wind_idx = find_ny_energy_row(df_generation, 'wind')
solar_fuels_idx = find_ny_energy_row(df_generation, 'solar')
hydro_idx = find_ny_energy_row(df_generation, 'conventional hydroelectric')

print(f"\nFound row indices:")
print(f"All fuels: {all_fuels_idx}")
print(f"Wind: {wind_idx}")
print(f"Solar: {solar_idx}")
print(f"Hydroelectric: {hydro_idx}")


Found row indices:
All fuels: 4
Wind: 7
Solar: 8
Hydroelectric: 5


In [37]:
print("\n" + "="*60)
print("STEP 2.2: RESHAPE DATA (WIDE TO LONG)")
print("="*60)

# Identify date columns (skip first 3: description, units, source key)
date_columns = df_generation.columns[:3].tolist()

print(f"Found {len(date_columns)} date columns")
print(f"First 5: {date_columns[:5]}")
print(f"Last 5: {date_columns[-5:]}")


STEP 2.2: RESHAPE DATA (WIDE TO LONG)
Found 3 date columns
First 5: ['description', 'units', 'source key']
Last 5: ['description', 'units', 'source key']


In [43]:
# Extract data for each energy source
energy_data = []

for name, idx in [('Total_Generation', all_fuels_idx), 
                   ('Hydroelectric', hydro_idx),
                   ('Wind', wind_idx), 
                   ('Solar', solar_idx)]:
    
    # Get the row
    row = df_generation.iloc[idx]
    
    # Extract values for date columns only
    values = row[date_columns].values
    
    # Create temporary dataframe
    temp_df = pd.DataFrame({
        'Date': date_columns,
        name: values
    })
    
    energy_data.append(temp_df)
    print(f"{name}: {len(values)} months extracted")

Total_Generation: 36 months extracted
Hydroelectric: 36 months extracted
Wind: 36 months extracted
Solar: 36 months extracted


In [44]:
# Merge all energy sources on Date
df_generation_clean = energy_data[0]
for df in energy_data[1:]:
    df_generation_clean = df_generation_clean.merge(df, on='Date', how='outer')

print(f"\nMerged data shape: {df_generation_clean.shape}")
print("\nFirst 10 rows:")
print(df_generation_clean.head(10))
print("\nData types:")
print(df_generation_clean.dtypes)


Merged data shape: (36, 5)

First 10 rows:
       Date Total_Generation Hydroelectric   Wind  Solar
0  Apr 2021           8733.0        2304.0  328.0  113.0
1  Apr 2022           9246.0        2182.0  443.0  166.0
2  Apr 2023           8293.0        2360.0  464.0  227.0
3  Aug 2021          12373.0        2439.0  212.0  109.0
4  Aug 2022          12326.0        2293.0  218.0  195.0
5  Aug 2023          11397.0        2460.0  302.0  243.0
6  Dec 2021          10293.0        2639.0  514.0   57.0
7  Dec 2022          10405.0        2328.0  520.0   97.0
8  Dec 2023          10415.0        2586.0  547.0   74.0
9  Feb 2021           9375.0        2074.0  318.0   46.0

Data types:
Date                object
Total_Generation    object
Hydroelectric       object
Wind                object
Solar               object
dtype: object


In [46]:
print("\n" + "="*60)
print("STEP 2.4: CLEAN DATE FORMAT & FILTER 2021-2023")
print("="*60)

# Convert Date column from "Jan 2021" format to datetime
df_generation_clean['Date'] = pd.to_datetime(
    df_generation_clean['Date'], 
    format='%b %Y',  # %b = abbreviated month name (Jan, Feb, etc.)
    errors='coerce'
)

print("Date conversion:")
print(f"Valid dates: {df_generation_clean['Date'].notna().sum()}")
print(f"Invalid dates: {df_generation_clean['Date'].isna().sum()}")
print(f"\nDate range: {df_generation_clean['Date'].min()} to {df_generation_clean['Date'].max()}")


STEP 2.4: CLEAN DATE FORMAT & FILTER 2021-2023
Date conversion:
Valid dates: 36
Invalid dates: 0

Date range: 2021-01-01 00:00:00 to 2023-12-01 00:00:00


In [47]:
# Filter for 2021-2023
df_generation_clean = df_generation_clean[
    (df_generation_clean['Date'] >= '2021-01-01') & 
    (df_generation_clean['Date'] <= '2023-12-31')
].copy()

# Sort by date
df_generation_clean = df_generation_clean.sort_values('Date').reset_index(drop=True)

print(f"\nAfter filtering - Shape: {df_generation_clean.shape}")
print("\nFirst 5 rows (chronological):")
print(df_generation_clean.head())
print("\nLast 5 rows:")
print(df_generation_clean.tail())


After filtering - Shape: (36, 5)

First 5 rows (chronological):
        Date Total_Generation Hydroelectric   Wind  Solar
0 2021-01-01          10773.0        2571.0  262.0   62.0
1 2021-02-01           9375.0        2074.0  318.0   46.0
2 2021-03-01           9909.0        2422.0  574.0  109.0
3 2021-04-01           8733.0        2304.0  328.0  113.0
4 2021-05-01           9333.0        2363.0  277.0  126.0

Last 5 rows:
         Date Total_Generation Hydroelectric   Wind  Solar
31 2023-08-01          11397.0        2460.0  302.0  243.0
32 2023-09-01          10113.0        2164.0  212.0  223.0
33 2023-10-01           9488.0        2240.0  466.0  160.0
34 2023-11-01          10412.0        2313.0  626.0  125.0
35 2023-12-01          10415.0        2586.0  547.0   74.0


In [49]:
print("\n" + "="*60)
print("STEP 2.5: UNIT CONVERSION (Thousand MWh → KWh)")
print("="*60)

# Current units: thousand megawatthours (1,000 MWh)
# Target units: kilowatthours (KWh)
# Conversion: 1,000 MWh = 1,000,000 KWh
# So: thousand MWh × 1,000,000 = KWh

print("Before conversion (first 5 rows):")
print(df_generation_clean.head())


STEP 2.5: UNIT CONVERSION (Thousand MWh → KWh)
Before conversion (first 5 rows):
        Date Total_Generation Hydroelectric   Wind  Solar
0 2021-01-01          10773.0        2571.0  262.0   62.0
1 2021-02-01           9375.0        2074.0  318.0   46.0
2 2021-03-01           9909.0        2422.0  574.0  109.0
3 2021-04-01           8733.0        2304.0  328.0  113.0
4 2021-05-01           9333.0        2363.0  277.0  126.0


In [50]:
# Convert all generation columns
for col in ['Total_Generation', 'Hydroelectric', 'Wind', 'Solar']:
    df_generation_clean[col] = df_generation_clean[col] * 1_000_000

print("\nAfter conversion to KWh (first 5 rows):")
print(df_generation_clean.head())

print("\nSample values comparison:")
print("Jan 2021 Total Generation: {:,.0f} KWh".format(df_generation_clean.iloc[0]['Total_Generation']))
print("Jan 2021 Wind: {:,.0f} KWh".format(df_generation_clean.iloc[0]['Wind']))


After conversion to KWh (first 5 rows):
        Date Total_Generation Hydroelectric         Wind        Solar
0 2021-01-01    10773000000.0  2571000000.0  262000000.0   62000000.0
1 2021-02-01     9375000000.0  2074000000.0  318000000.0   46000000.0
2 2021-03-01     9909000000.0  2422000000.0  574000000.0  109000000.0
3 2021-04-01     8733000000.0  2304000000.0  328000000.0  113000000.0
4 2021-05-01     9333000000.0  2363000000.0  277000000.0  126000000.0

Sample values comparison:
Jan 2021 Total Generation: 10,773,000,000 KWh
Jan 2021 Wind: 262,000,000 KWh


In [52]:
print("\n" + "="*60)
print("STEP 2.6: CHECK MISSING VALUES & SAVE")
print("="*60)

# Check for missing values
print("Missing values by column:")
print(df_generation_clean.isnull().sum())


STEP 2.6: CHECK MISSING VALUES & SAVE
Missing values by column:
Date                0
Total_Generation    0
Hydroelectric       0
Wind                0
Solar               0
dtype: int64


In [56]:
# Check for any zero or negative values (shouldn't exist in generation data)
print("\nChecking for zero or negative values:")
for col in ['Total_Generation', 'Hydroelectric', 'Wind', 'Solar']:
    zero_count = (df_generation_clean[col] == 0).sum()
    neg_count = (df_generation_clean[col] < 0).sum()
    if zero_count > 0 or neg_count > 0:
        print(f"{col}: {zero_count} zeros, {neg_count} negatives")
    else:
        print(f"{col}: OK")

# Summary statistics
print("\nSummary statistics:")
print(df_generation_clean.describe())

# Save cleaned data
output_path = '../data/processed/generation_clean.csv'
df_generation_clean.to_csv(output_path, index=False)
print(f"Saved to: {output_path}")


Checking for zero or negative values:
Total_Generation: OK
Hydroelectric: OK
Wind: OK
Solar: OK

Summary statistics:
                      Date
count                   36
mean   2022-06-16 12:00:00
min    2021-01-01 00:00:00
25%    2021-09-23 12:00:00
50%    2022-06-16 00:00:00
75%    2023-03-08 18:00:00
max    2023-12-01 00:00:00
Saved to: ../data/processed/generation_clean.csv


In [63]:
print("\n" + "="*60)
print("3. LOADING NOAA WEATHER DATA")
print("="*60)

df_weather = pd.read_csv('../data/raw/noaa_weather.csv')

print(f"\nOriginal data shape: {df_weather.shape}")
print(f"\nColumns: {list(df_weather.columns)}")
print("\nFirst 10 rows:")
print(df_weather.head(10))

# Check data types
print("\nData types:")
print(df_weather.dtypes)


3. LOADING NOAA WEATHER DATA

Original data shape: (1095, 15)

Columns: ['STATION', 'NAME', 'DATE', 'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5']

First 10 rows:
       STATION                         NAME        DATE  AWND  PGTM  PRCP  \
0  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-01  5.59   NaN  0.62   
1  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-02  6.71   NaN  0.05   
2  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-03  9.17   NaN  0.22   
3  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-04  4.47   NaN  0.02   
4  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-05  4.92   NaN  0.00   
5  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-06  8.28   NaN  0.00   
6  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-07  6.93   NaN  0.00   
7  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-08  6.04   NaN  0.00   
8  USW00094728  NY CITY CENTRAL PARK, NY US  2021-01-09  6.93   NaN  0.00   
9  USW00094728  N

In [65]:
# Check for key columns
required_cols = ['DATE', 'TMAX', 'TMIN', 'PRCP', 'AWND']
missing_cols = [col for col in required_cols if col not in df_weather.columns]
if missing_cols:
    print(f"\nWARNING: Missing columns: {missing_cols}")
else:
    print("\nAll required columns present")


All required columns present


In [68]:
print("\n" + "="*60)
print("STEP 3.1: CALCULATE TAVG & CLEAN DATA")
print("="*60)

# Convert DATE to datetime
df_weather['DATE'] = pd.to_datetime(df_weather['DATE'], errors='coerce')

print(f"Date range: {df_weather['DATE'].min()} to {df_weather['DATE'].max()}")


STEP 3.1: CALCULATE TAVG & CLEAN DATA
Date range: 2021-01-01 00:00:00 to 2023-12-31 00:00:00


In [69]:
# Calculate TAVG from TMAX and TMIN (TAVG column is empty)
df_weather['TAVG'] = (df_weather['TMAX'] + df_weather['TMIN']) / 2

print("\nTAVG calculation:")
print(f"TMAX available: {df_weather['TMAX'].notna().sum()} days")
print(f"TMIN available: {df_weather['TMIN'].notna().sum()} days")
print(f"TAVG calculated: {df_weather['TAVG'].notna().sum()} days")


TAVG calculation:
TMAX available: 1095 days
TMIN available: 1095 days
TAVG calculated: 1095 days


In [71]:
# Select only needed columns
weather_cols = ['DATE', 'TAVG', 'PRCP', 'AWND']
df_weather_clean = df_weather[weather_cols].copy()

# Filter for 2021-2023
df_weather_clean = df_weather_clean[
    (df_weather_clean['DATE'] >= '2021-01-01') &
    (df_weather_clean['DATE'] <= '2023-12-31')
].copy()

print(f"\nAfter filtering 2021-2023 - Shape: {df_weather_clean.shape}")
print("\nFirst 10 rows:")
print(df_weather_clean.head(10))
print("\nMissing values:")
print(df_weather_clean.isnull().sum())


After filtering 2021-2023 - Shape: (1095, 4)

First 10 rows:
        DATE  TAVG  PRCP  AWND
0 2021-01-01  36.5  0.62  5.59
1 2021-01-02  43.5  0.05  6.71
2 2021-01-03  36.0  0.22  9.17
3 2021-01-04  39.0  0.02  4.47
4 2021-01-05  39.0  0.00  4.92
5 2021-01-06  37.5  0.00  8.28
6 2021-01-07  35.5  0.00  6.93
7 2021-01-08  32.0  0.00  6.04
8 2021-01-09  30.5  0.00  6.93
9 2021-01-10  35.0  0.00  4.25

Missing values:
DATE     0
TAVG     0
PRCP     0
AWND    18
dtype: int64


In [75]:
print("\n" + "="*60)
print("STEP 3.2: AGGREGATE TO MONTHLY AVERAGES")
print("="*60)

# Extract year-month for grouping
df_weather_clean['Year'] = df_weather_clean['DATE'].dt.year
df_weather_clean['Month'] = df_weather_clean['DATE'].dt.month

# Create date column (first day of each month)
df_weather_clean['Date'] = pd.to_datetime(
    df_weather_clean['Year'].astype(str) + '-' + 
    df_weather_clean['Month'].astype(str) + '-01'
)

# Group by month and calculate averages
df_weather_monthly = df_weather_clean.groupby('Date').agg({
    'TAVG': 'mean',
    'PRCP': 'sum',    # Total precipitation per month
    'AWND': 'mean'
}).reset_index()

# Rename columns
df_weather_monthly.columns = ['Date', 'Avg_Temp', 'Total_Precip', 'Avg_Wind']

print(f"\nMonthly aggregated shape: {df_weather_monthly.shape}")
print("\nFirst 10 months:")
print(df_weather_monthly.head(10))
print("\nLast 5 months:")
print(df_weather_monthly.tail())
print("\nMissing values:")
print(df_weather_monthly.isnull().sum())


STEP 3.2: AGGREGATE TO MONTHLY AVERAGES

Monthly aggregated shape: (36, 4)

First 10 months:
        Date   Avg_Temp  Total_Precip  Avg_Wind
0 2021-01-01  34.758065          2.31  6.797419
1 2021-02-01  34.160714          5.13  6.487143
2 2021-03-01  45.822581          3.41  6.710323
3 2021-04-01  54.583333          2.69  5.741667
4 2021-05-01  62.887097          4.36  5.174194
5 2021-06-01  74.250000          2.62  3.682667
6 2021-07-01  76.032258         11.09  3.485484
7 2021-08-01  77.548387         10.32  3.441613
8 2021-09-01  70.300000         10.03  3.974118
9 2021-10-01  62.000000          5.26  4.621481

Last 5 months:
         Date   Avg_Temp  Total_Precip  Avg_Wind
31 2023-08-01  75.000000          6.56  3.722903
32 2023-09-01  69.400000         14.25  4.637333
33 2023-10-01  60.500000          3.90  3.824194
34 2023-11-01  46.666667          2.95  4.868667
35 2023-12-01  44.612903          6.71  5.109677

Missing values:
Date            0
Avg_Temp        0
Total_Precip   

In [77]:
print("\n" + "="*60)
print("STEP 3.3: SAVE WEATHER DATA")
print("="*60)

# Save to processed folder
output_path = '../data/processed/weather_clean.csv'
df_weather_monthly.to_csv(output_path, index=False)

print(f"Saved to: {output_path}")
print(df_weather_monthly.describe())


STEP 3.3: SAVE WEATHER DATA
Saved to: ../data/processed/weather_clean.csv
                      Date   Avg_Temp  Total_Precip   Avg_Wind
count                   36  36.000000     36.000000  36.000000
mean   2022-06-16 12:00:00  57.022746      4.592222   4.977272
min    2021-01-01 00:00:00  30.322581      1.120000   2.597097
25%    2021-09-23 12:00:00  44.625000      2.672500   3.854167
50%    2022-06-16 00:00:00  57.243548      4.195000   4.866108
75%    2023-03-08 18:00:00  70.075000      5.280000   6.011630
max    2023-12-01 00:00:00  79.548387     14.250000   6.797419
std                    NaN  14.965058      2.982224   1.181608


In [78]:
print("\n" + "="*60)
print("4. MERGING ALL DATASETS")
print("="*60)

# Load all cleaned datasets
df_consumption_clean = pd.read_csv('../data/processed/consumption_clean.csv')
df_generation_clean = pd.read_csv('../data/processed/generation_clean.csv')
df_weather_monthly = pd.read_csv('../data/processed/weather_clean.csv')

# Convert Date columns to datetime
df_consumption_clean['Date'] = pd.to_datetime(df_consumption_clean['Date'])
df_generation_clean['Date'] = pd.to_datetime(df_generation_clean['Date'])
df_weather_monthly['Date'] = pd.to_datetime(df_weather_monthly['Date'])

print("Dataset shapes:")
print(f"Consumption: {df_consumption_clean.shape}")
print(f"Generation: {df_generation_clean.shape}")
print(f"Weather: {df_weather_monthly.shape}")

# Check date ranges
print("\nDate ranges:")
print(f"Consumption: {df_consumption_clean['Date'].min()} to {df_consumption_clean['Date'].max()}")
print(f"Generation: {df_generation_clean['Date'].min()} to {df_generation_clean['Date'].max()}")
print(f"Weather: {df_weather_monthly['Date'].min()} to {df_weather_monthly['Date'].max()}")


4. MERGING ALL DATASETS
Dataset shapes:
Consumption: (36, 5)
Generation: (36, 5)
Weather: (36, 4)

Date ranges:
Consumption: 2021-01-01 00:00:00 to 2023-12-01 00:00:00
Generation: 2021-01-01 00:00:00 to 2023-12-01 00:00:00
Weather: 2021-01-01 00:00:00 to 2023-12-01 00:00:00


In [95]:
print("\n" + "="*60)
print("STEP 4.1: MERGE ALL DATASETS")
print("="*60)

# Reload all cleaned datasets
df_consumption_clean = pd.read_csv('../data/processed/consumption_clean.csv')
df_generation_clean = pd.read_csv('../data/processed/generation_clean.csv')
df_weather_monthly = pd.read_csv('../data/processed/weather_clean.csv')

# Convert dates
df_consumption_clean['Date'] = pd.to_datetime(df_consumption_clean['Date'])
df_generation_clean['Date'] = pd.to_datetime(df_generation_clean['Date'])
df_weather_monthly['Date'] = pd.to_datetime(df_weather_monthly['Date'])

print("Before merge:")
print(f"Consumption: {df_consumption_clean.shape}")
print(f"Generation: {df_generation_clean.shape}")
print(f"Weather: {df_weather_monthly.shape}")


STEP 4.1: MERGE ALL DATASETS
Before merge:
Consumption: (252, 3)
Generation: (36, 5)
Weather: (36, 4)


In [96]:
# Merge Generation + Weather first
df_gen_weather = df_generation_clean.merge(df_weather_monthly, on='Date', how='inner')
print(f"\nGen+Weather: {df_gen_weather.shape}")

# Merge with Consumption
df_merged = df_consumption_clean.merge(df_gen_weather, on='Date', how='inner')

print(f"\nFinal merged: {df_merged.shape}")
print("\nColumns:")
print(df_merged.columns.tolist())
print("\nFirst 10 rows:")
print(df_merged.head(10))


Gen+Weather: (36, 8)

Final merged: (252, 10)

Columns:
['Borough', 'Date', 'Consumption_KWH', 'Total_Generation', 'Hydroelectric', 'Wind', 'Solar', 'Avg_Temp', 'Total_Precip', 'Avg_Wind']

First 10 rows:
  Borough       Date  Consumption_KWH  Total_Generation  Hydroelectric  \
0   BRONX 2021-01-01       24747982.0      1.077300e+10   2.571000e+09   
1   BRONX 2021-02-01       21503857.0      9.375000e+09   2.074000e+09   
2   BRONX 2021-03-01       20692175.0      9.909000e+09   2.422000e+09   
3   BRONX 2021-04-01       21424289.0      8.733000e+09   2.304000e+09   
4   BRONX 2021-05-01       20748428.0      9.333000e+09   2.363000e+09   
5   BRONX 2021-06-01       26845332.0      1.109700e+10   2.337000e+09   
6   BRONX 2021-07-01       33006942.0      1.167900e+10   2.429000e+09   
7   BRONX 2021-08-01       36462677.0      1.237300e+10   2.439000e+09   
8   BRONX 2021-09-01       29853273.0      9.893000e+09   2.267000e+09   
9   BRONX 2021-10-01       23251802.0      9.574000e+0

In [101]:
print("\n" + "="*60)
print("STEP 4.2: SAVE MERGED DATA")
print("="*60)

# Check for missing values
print("Missing values:")
print(df_merged.isnull().sum())

# Summary
print("\nFinal dataset summary:")
print(f"Shape: {df_merged.shape}")
print(f"Date range: {df_merged['Date'].min()} to {df_merged['Date'].max()}")
print(f"Boroughs: {df_merged['Borough'].unique().tolist()}")

# Save merged dataset
output_path = '../data/processed/merged_data.csv'
df_merged.to_csv(output_path, index=False)

print(f"\n✓ Saved to: {output_path}")


STEP 4.2: SAVE MERGED DATA
Missing values:
Borough             0
Date                0
Consumption_KWH     0
Total_Generation    0
Hydroelectric       0
Wind                0
Solar               0
Avg_Temp            0
Total_Precip        0
Avg_Wind            0
dtype: int64

Final dataset summary:
Shape: (252, 10)
Date range: 2021-01-01 00:00:00 to 2023-12-01 00:00:00
Boroughs: ['BRONX', 'BROOKLYN', 'FHA', 'MANHATTAN', 'NON DEVELOPMENT FACILITY', 'QUEENS', 'STATEN ISLAND']

✓ Saved to: ../data/processed/merged_data.csv
